In [91]:
using Plots
using PyCall
using ControlSystems

gr()

Plots.GRBackend()

In [122]:
zc=0.8
g=9.8
dt=0.001

#支持時間
Tsup=1
#全体のステップ数
k_max=Int32(1+(Tsup*6)/dt)
#目標予見ステップ
M=1000

#支持脚位置
sx=[0.0 0.0 0.3 0.6 0.9 0.9]
sy=[0.0 0.2 0.0 0.2 0.0 0.2]


#目標ZMPベクトル
px_ref=[[sx[1] for s in 1:Tsup*1000]
        [sx[2] for s in 1:Tsup*1000]
        [sx[3] for s in 1:Tsup*1000]
        [sx[4] for s in 1:Tsup*1000]
        [sx[5] for s in 1:Tsup*1000]
        [sx[6] for s in 1:Tsup*1001+M]]

py_ref=[[sy[1] for s in 1:Tsup*1000]
        [sy[2] for s in 1:Tsup*1000]
        [sy[3] for s in 1:Tsup*1000]
        [sy[4] for s in 1:Tsup*1000]
        [sy[5] for s in 1:Tsup*1000]
        [sy[6] for s in 1:Tsup*1001+M]]

#計算上のZMPベクトル
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max) 

6001-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [138]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[1000 0 0 0
    0 10 0 0
    0 0 1 0
    0 0 0 100]
R=1.0

#エラーシステム

#内部状態ベクトル
x=[0.0
    0.0
    0.0]

y=[0.01
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0]

4-element Array{Float64,1}:
 1.0
 0.0
 0.0
 0.0

In [139]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #plotするために記録
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #plotするために記録
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 


xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y


dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR


old_x=x
old_y=y

xu=0
yu=0

P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 -0.357332

In [140]:
for i in 1:k_max-1  
    #求めた入力を用いてk+1の状態を計算する
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]
    
    k+=1
    
    x_plot_log[k+1]=x[1] #plotするために記録
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #plotするために記録
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    
    dx=x-old_x
    dy=y-old_y
    
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    
    
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y
    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    
    old_x=x
    old_y=y
    
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end
    

In [141]:
plot(t,px_ref)
plot!(t,px)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
<polyline clip-path="url(#clip0102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,362.773 39.4622,362.773 39.5543,362.773 39.6465,362.773 39.7386,362.773 39.8307,362.773 39.9228,362.773 40.015,362.773 40.1071,362.773 40.1992,362.773 
 40.2913,362.773 40.3835,362.773 40.4756,362.773 40.5677,362.773 40.6598,362.773 40.752,362.773 40.8441,362.773 40.9362,362.773 41.0283,362.773 41.1205,362.773 
 41.2126,362.773 41.3047,362.773 41.3969,362.773 41.489,362.773 41.5811,362.773 41.6732,362.773 41.7654,362.773 41.8575,362.773 41.9496,362.773 42.0417,362.773 
 42.1339,362.773 42.226,362.773 42.3181,362.773 42.4102,362.773 42.5024,362.773 42.5945,362.773 42.6866,362.773 42.7787,362.773 42.8709,362.773 42.963,362.773 
 43.0551,362.773 43.1472,362.773 43.2394,362.773 43.3315,362.773 43.4236,362.773 43.5157,362.773 43.6079,362.773 43.7,362.773 43.7921,362.773 43.8843,362.773 
 43.9764,362.773 44.0685,362.773 44.1606,362.773 44.2528,362.773 44.3449,362.773 44.437,362.773 44.5291,362.773 44.6213,362.773 44.7134,362.773 44.8055,362.773 
 44.8976,362.773 44.9898,362.773 45.0819,362.773 45.174,362.773 45.2661,362.773 45.3583,362.773 45.4504,362.773 45.5425,362.773 45.6346,362.773 45.7268,362.773 
 45.8189,362.773 45.911,362.773 46.0031,362.773 46.0953,362.773 46.1874,362.773 46.2795,362.773 46.3717,362.773 46.4638,362.773 46.5559,362.773 46.648,362.773 
 46.7402,362.773 46.8323,362.773 46.9244,362.773 47.0165,362.773 47.1087,362.773 47.2008,362.773 47.2929,362.773 47.385,362.773 47.4772,362.773 47.5693,362.773 
 47.6614,362.773 47.7535,362.773 47.8457,362.773 47.9378,362.773 48.0299,362.773 48.122,362.773 48.2142,362.773 48.3063,362.773 48.3984,362.773 48.4906,362.773 
 48.5827,362.773 48.6748,362.773 48.7669,362.773 48.8591,362.773 48.9512,362.773 49.0433,362.773 49.1354,362.773 49.2276,362.773 49.3197,362.773 49.4118,362.773 
 49.5039,362.773 49.5961,362.773 49.6882,362.773 49.7803,362.773 49.8724,362.773 49.9646,362.773 50.0567,362.773 50.1488,362.773 50.2409,362.773 50.3331,362.773 
 50.4252,362.773 50.5173,362.773 50.6094,362.773 50.7016,362.773 50.7937,362.773 50.8858,362.773 50.978,362.773 51.0701,362.773 51.1622,362.773 51.2543,362.773 
 51.3465,362.773 51.4386,362.773 51.5307,362.773 51.6228,362.773 51.715,362.773 51.8071,362.773 51.8992,362.773 51.9913,362.773 52.0835,362.773 52.1756,362.773 
 52.2677,362.773 52.3598,362.773 52.452,362.773 52.5441,362.773 52.6362,362.773 52.7283,362.773 52.8205,362.773 52.9126,362.773 53.0047,362.773 53.0969,362.773 
 53.189,362.773 53.2811,362.773 53.3732,362.773 53.4654,362.773 53.5575,362.773 53.6496,362.773 53.7417,362.773 53.8339,362.773 53.926,362.773 54.0181,362.773 
 54.1102,362.773 54.2024,362.773 54.2945,362.773 54.3866,362.773 54.4787,362.773 54.5709,362.773 54.663,362.773 54.7551,362.773 54.8472,362.773 54.9394,362.773 
 55.0315,362.773 55.1236,362.773 55.2157,362.773 55.3079,362.773 55.4,362.773 55.4921,362.773 55.5843,362.773 55.6764,362.773 55.7685,362.773 55.8606,362.773 
 55.9528,362.773 56.0449,362.773 56.137,362.773 56.2291,362.773 56.3213,362.773 56.4134,362.773 56.5055,362.773 56.5976,362.773 56.6898,362.773 56.7819,362.773 
 56.874,362.773 56.9661,362.773 57.0583,362.773 57.1504,362.773 57.2425,362.773 57.3346,362.773 57.4268,362.773 57.5189,362.773 57.611,362.773 57.7031,362.773 
 57.7953,362.773 57.8874,362.773 57.9795,362.773 58.0717,362.773 58.1638,362.773 58.2559,362.773 58.348,362.773 58.4402,362.773 58.5323,362.773 58.6244,362.773 
 58.7165,362.773 58.8087,362.773 58.9008,362.773 58.9929,362.773 59.085,362.773 59.1772,362.773 59.2693,362.773 59.3614,362.773 59.4535,362.773 59.5457,362.773 
 59.6378,362.773 59.7299,362.773 59.822,362.773 59.9142,362.773 60.0063,362.773 60.0984,362.773 60.1906,362.773

In [142]:
plot(t,py_ref)
plot!(t,py)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
<polyline clip-path="url(#clip0202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,364.319 39.4622,364.319 39.5543,364.319 39.6465,364.319 39.7386,364.319 39.8307,364.319 39.9228,364.319 40.015,364.319 40.1071,364.319 40.1992,364.319 
 40.2913,364.319 40.3835,364.319 40.4756,364.319 40.5677,364.319 40.6598,364.319 40.752,364.319 40.8441,364.319 40.9362,364.319 41.0283,364.319 41.1205,364.319 
 41.2126,364.319 41.3047,364.319 41.3969,364.319 41.489,364.319 41.5811,364.319 41.6732,364.319 41.7654,364.319 41.8575,364.319 41.9496,364.319 42.0417,364.319 
 42.1339,364.319 42.226,364.319 42.3181,364.319 42.4102,364.319 42.5024,364.319 42.5945,364.319 42.6866,364.319 42.7787,364.319 42.8709,364.319 42.963,364.319 
 43.0551,364.319 43.1472,364.319 43.2394,364.319 43.3315,364.319 43.4236,364.319 43.5157,364.319 43.6079,364.319 43.7,364.319 43.7921,364.319 43.8843,364.319 
 43.9764,364.319 44.0685,364.319 44.1606,364.319 44.2528,364.319 44.3449,364.319 44.437,364.319 44.5291,364.319 44.6213,364.319 44.7134,364.319 44.8055,364.319 
 44.8976,364.319 44.9898,364.319 45.0819,364.319 45.174,364.319 45.2661,364.319 45.3583,364.319 45.4504,364.319 45.5425,364.319 45.6346,364.319 45.7268,364.319 
 45.8189,364.319 45.911,364.319 46.0031,364.319 46.0953,364.319 46.1874,364.319 46.2795,364.319 46.3717,364.319 46.4638,364.319 46.5559,364.319 46.648,364.319 
 46.7402,364.319 46.8323,364.319 46.9244,364.319 47.0165,364.319 47.1087,364.319 47.2008,364.319 47.2929,364.319 47.385,364.319 47.4772,364.319 47.5693,364.319 
 47.6614,364.319 47.7535,364.319 47.8457,364.319 47.9378,364.319 48.0299,364.319 48.122,364.319 48.2142,364.319 48.3063,364.319 48.3984,364.319 48.4906,364.319 
 48.5827,364.319 48.6748,364.319 48.7669,364.319 48.8591,364.319 48.9512,364.319 49.0433,364.319 49.1354,364.319 49.2276,364.319 49.3197,364.319 49.4118,364.319 
 49.5039,364.319 49.5961,364.319 49.6882,364.319 49.7803,364.319 49.8724,364.319 49.9646,364.319 50.0567,364.319 50.1488,364.319 50.2409,364.319 50.3331,364.319 
 50.4252,364.319 50.5173,364.319 50.6094,364.319 50.7016,364.319 50.7937,364.319 50.8858,364.319 50.978,364.319 51.0701,364.319 51.1622,364.319 51.2543,364.319 
 51.3465,364.319 51.4386,364.319 51.5307,364.319 51.6228,364.319 51.715,364.319 51.8071,364.319 51.8992,364.319 51.9913,364.319 52.0835,364.319 52.1756,364.319 
 52.2677,364.319 52.3598,364.319 52.452,364.319 52.5441,364.319 52.6362,364.319 52.7283,364.319 52.8205,364.319 52.9126,364.319 53.0047,364.319 53.0969,364.319 
 53.189,364.319 53.2811,364.319 53.3732,364.319 53.4654,364.319 53.5575,364.319 53.6496,364.319 53.7417,364.319 53.8339,364.319 53.926,364.319 54.0181,364.319 
 54.1102,364.319 54.2024,364.319 54.2945,364.319 54.3866,364.319 54.4787,364.319 54.5709,364.319 54.663,364.319 54.7551,364.319 54.8472,364.319 54.9394,364.319 
 55.0315,364.319 55.1236,364.319 55.2157,364.319 55.3079,364.319 55.4,364.319 55.4921,364.319 55.5843,364.319 55.6764,364.319 55.7685,364.319 55.8606,364.319 
 55.9528,364.319 56.0449,364.319 56.137,364.319 56.2291,364.319 56.3213,364.319 56.4134,364.319 56.5055,364.319 56.5976,364.319 56.6898,364.319 56.7819,364.319 
 56.874,364.319 56.9661,364.319 57.0583,364.319 57.1504,364.319 57.2425,364.319 57.3346,364.319 57.4268,364.319 57.5189,364.319 57.611,364.319 57.7031,364.319 
 57.7953,364.319 57.8874,364.319 57.9795,364.319 58.0717,364.319 58.1638,364.319 58.2559,364.319 58.348,364.319 58.4402,364.319 58.5323,364.319 58.6244,364.319 
 58.7165,364.319 58.8087,364.319 58.9008,364.319 58.9929,364.319 59.085,364.319 59.1772,364.319 59.2693,364.319 59.3614,364.319 59.4535,364.319 59.5457,364.319 
 59.6378,364.319 59.7299,364.319 59.822,364.319 59.9142,364.319 60.0063,364.319 60.0984,364.319 60.1906,36

In [143]:
plot(x_plot_log,y_plot_log)
plot!(0,1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
<polyline clip-path="url(#clip0302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 
 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 
 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 39.3701,368.504 
 39.3701,368.503 39.3701,368.503 39.3701,368.503 39.3701,368.503 39.3701,368.503 39.3701,368.503 39.3701,368.502 39.3701,368.502 39.3701,368.502 39.3701,368.502 
 39.3701,368.501 39.3701,368.501 39.3701,368.501 39.3701,368.5 39.3701,368.5 39.3701,368.499 39.3701,368.499 39.3701,368.498 39.3701,368.498 39.3701,368.497 
 39.3701,368.497 39.3701,368.496 39.3701,368.495 39.3701,368.494 39.3701,368.494 39.3701,368.493 39.3701,368.492 39.3701,368.491 39.3701,368.49 39.3701,368.489 
 39.3701,368.488 39.3701,368.487 39.3701,368.486 39.3701,368.484 39.3701,368.483 39.3701,368.482 39.3701,368.48 39.3701,368.479 39.3701,368.478 39.3701,368.476 
 39.3701,368.474 39.3701,368.473 39.3701,368.471 39.3701,368.469 39.3701,368.468 39.3701,368.466 39.3701,368.464 39.3701,368.462 39.3701,368.46 39.3701,368.458 
 39.3701,368.455 39.3701,368.453 39.3701,368.451 39.3701,368.448 39.3701,368.446 39.3701,368.443 39.3701,368.441 39.3701,368.438 39.3701,368.435 39.3701,368.433 
 39.3701,368.43 39.3701,368.427 39.3701,368.424 39.3701,368.421 39.3701,368.418 39.3701,368.414 39.3701,368.411 39.3701,368.408 39.3701,368.404 39.3701,368.401 
 39.3701,368.397 39.3701,368.393 39.3701,368.39 39.3701,368.386 39.3701,368.382 39.3701,368.378 39.3701,368.374 39.3701,368.37 39.3701,368.365 39.3701,368.361 
 39.3701,368.356 39.3701,368.352 39.3701,368.347 39.3701,368.343 39.3701,368.338 39.3701,368.333 39.3701,368.328 39.3701,368.323 39.3701,368.318 39.3701,368.313 
 39.3701,368.307 39.3701,368.302 39.3701,368.297 39.3701,368.291 39.3701,368.285 39.3701,368.28 39.3701,368.274 39.3701,368.268 39.3701,368.262 39.3701,368.256 
 39.3701,368.249 39.3701,368.243 39.3701,368.236 39.3701,368.23 39.3701,368.223 39.3701,368.217 39.3701,368.21 39.3701,368.203 39.3701,368.196 39.3701,368.189 
 39.3701,368.181 39.3701,368.174 39.3701,368.167 39.3701,368.159 39.3701,368.151 39.3701,368.144 39.3701,368.136 39.3701,368.128 39.3701,368.12 39.3701,368.112 
 39.3701,368.103 39.3701,368.095 39.3701,368.086 39.3701,368.078 39.3701,368.069 39.3701,368.06 39.3701,368.051 39.3701,368.042 39.3701,368.033 39.3701,368.024 
 39.3701,368.014 39.3701,368.005 39.3701,367.995 39.3701,367.985 39.3701,367.976 39.3701,367.966 39.3701,367.955 39.3701,367.945 39.3701,367.935 39.3701,367.924 
 39.3701,367.914 39.3701,367.903 39.3701,367.892 39.3701,367.881 39.3701,367.87 39.3701,367.859 39.3701,367.848 39.3701,367.836 39.3701,367.825 39.3701,367.813 
 39.3701,367.801 39.3701,367.789 39.3701,367.777 39.3701,367.765 39.3701,367.753 39.3701,367.74 39.3701,367.728 39.3701,367.715 39.3701,367.702 39.3701,367.689 
 39.3701,367.676 39.3701,367.663 39.3701,367.65 39.3701,367.636 39.3701,367.623 39.3701,367.609 39.3701,367.595 39.3701,367.581 39.3701,367.567 39.3701,367.553 
 39.3701,367.538 39.3701,367.523 39.3701,367.509 39.3701,367.494 39.3701,367.479 39.3701,367.464 39.3701,367.449 39.3701,367.433 39.3701,367.418 39.3701,367.402 
 39.3701,367.386 39.3701,367.37 39.3701,367.354 39.3701,367.338 39.3701,367.321 39.3701,367.305 39.3701,367.288 39.3701,367.271 39.3701,367.254 39.3701,367.237 
 39.3701,367.22 39.3701,367.202 39.3701,367.185 39.3701,367.167 39.3701,367.149 

In [144]:
plot(t,yv_plot_log)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1000 
 
 
 2000 
 
 
 3000 
 
 
 4000 
 
 
 5000 
 
 
 6000 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip0402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,193.42 39.4622,193.42 39.5543,193.42 39.6465,193.421 39.7386,193.421 39.8307,193.422 39.9228,193.422 40.015,193.423 40.1071,193.424 40.1992,193.424 
 40.2913,193.425 40.3835,193.425 40.4756,193.425 40.5677,193.425 40.6598,193.425 40.752,193.424 40.8441,193.424 40.9362,193.423 41.0283,193.422 41.1205,193.42 
 41.2126,193.418 41.3047,193.416 41.3969,193.414 41.489,193.411 41.5811,193.408 41.6732,193.405 41.7654,193.401 41.8575,193.397 41.9496,193.392 42.0417,193.387 
 42.1339,193.382 42.226,193.376 42.3181,193.37 42.4102,193.364 42.5024,193.357 42.5945,193.35 42.6866,193.342 42.7787,193.335 42.8709,193.326 42.963,193.318 
 43.0551,193.308 43.1472,193.299 43.2394,193.289 43.3315,193.279 43.4236,193.268 43.5157,193.257 43.6079,193.246 43.7,193.234 43.7921,193.222 43.8843,193.21 
 43.9764,193.197 44.0685,193.184 44.1606,193.17 44.2528,193.157 44.3449,193.142 44.437,193.128 44.5291,193.113 44.6213,193.098 44.7134,193.082 44.8055,193.067 
 44.8976,193.05 44.9898,193.034 45.0819,193.017 45.174,193 45.2661,192.983 45.3583,192.965 45.4504,192.947 45.5425,192.929 45.6346,192.91 45.7268,192.891 
 45.8189,192.872 45.911,192.853 46.0031,192.833 46.0953,192.813 46.1874,192.793 46.2795,192.772 46.3717,192.751 46.4638,192.73 46.5559,192.709 46.648,192.687 
 46.7402,192.666 46.8323,192.644 46.9244,192.621 47.0165,192.599 47.1087,192.576 47.2008,192.553 47.2929,192.529 47.385,192.506 47.4772,192.482 47.5693,192.458 
 47.6614,192.434 47.7535,192.409 47.8457,192.384 47.9378,192.359 48.0299,192.334 48.122,192.309 48.2142,192.283 48.3063,192.257 48.3984,192.231 48.4906,192.205 
 48.5827,192.178 48.6748,192.151 48.7669,192.124 48.8591,192.097 48.9512,192.07 49.0433,192.042 49.1354,192.014 49.2276,191.986 49.3197,191.958 49.4118,191.93 
 49.5039,191.901 49.5961,191.872 49.6882,191.843 49.7803,191.814 49.8724,191.784 49.9646,191.754 50.0567,191.724 50.1488,191.694 50.2409,191.664 50.3331,191.633 
 50.4252,191.602 50.5173,191.572 50.6094,191.54 50.7016,191.509 50.7937,191.477 50.8858,191.446 50.978,191.414 51.0701,191.381 51.1622,191.349 51.2543,191.316 
 51.3465,191.284 51.4386,191.251 51.5307,191.217 51.6228,191.184 51.715,191.15 51.8071,191.117 51.8992,191.083 51.9913,191.048 52.0835,191.014 52.1756,190.979 
 52.2677,190.945 52.3598,190.91 52.452,190.874 52.5441,190.839 52.6362,190.804 52.7283,190.768 52.8205,190.732 52.9126,190.696 53.0047,190.659 53.0969,190.623 
 53.189,190.586 53.2811,190.549 53.3732,190.512 53.4654,190.474 53.5575,190.437 53.6496,190.399 53.7417,190.361 53.8339,190.323 53.926,190.284 54.0181,190.246 
 54.1102,190.207 54.2024,190.168 54.2945,190.129 54.3866,190.089 54.4787,190.05 54.5709,190.01 54.663,189.97 54.7551,189.93 54.8472,189.89 54.9394,189.849 
 55.0315,189.808 55.1236,189.767 55.2157,189.726 55.3079,189.685 55.4,189.643 55.4921,189.602 55.5843,189.56 55.6764,189.518 55.7685,189.475 55.8606,189.433 
 55.9528,189.39 56.0449,189.347 56.137,189.304 56.2291,189.261 56.3213,189.217 56.4134,189.173 56.5055,189.129 56.5976,189.085 56.6898,189.041 56.7819,188.996 
 56.874,188.952 56.9661,188.907 57.0583,188.862 57.1504,188.816 57.2425,188.771 57.3346,188.725 57.4268,188.679 57.5189,188.633 57.611,188.587 57.7031,188.54 
 57.7953,188.493 57.8874,188.447 57.9795,188.399 58.0717,188.352 58.1638,188.305 58.2559,188.257 58.348,188.209 58.4402,188.161 58.5323,188.113 58.6244,188.064 
 58.7165,188.015 58.8087,187.966 58.9008,187.917 58.9929,187.868 59.085,187.818 59.1772,187.769 59.2693,187.719 59.3614,187.669 59.4535,187.618 59.5457,187.568 
 59.6378,187.517 59.7299,187.466 59.822,187.415 59.9142,187.364 60.0063,187.312 60.0984,187.26 60.1906,187.208 60.2827,187.156 60.3748,18